In [18]:
import numpy as np
import sys 
import os
sys.path.append('/home/giorgiod/Documents/APDFT/dep')
sys.path.append('/home/giorgiod/Documents/APDFT/src')

In [19]:
from apdft import Calculator

In [20]:
calc_path = '/home/giorgiod/MRCC_interface/Diatomics/from_co_opt_ccsd_ccpvtz_68_to_77_A5_G5/A_2_G_0/'
proj_path= '/home/giorgiod/MRCC_interface/Diatomics/from_co_opt_ccsd_ccpvtz_68_to_77_A5_G5/'
grid = Calculator.MrccCalculator.get_grid(None, None, calc_path)

In [21]:
print (type(grid))
grid[0].shape, grid[1].shape #grid=(array[[x0,y0,z0],...,[xN,yN,zN]], array[w0,...wN])

<class 'tuple'>


((58060, 3), (58060,))

In [26]:
ccdensity = Calculator.MrccCalculator.density_on_grid(calc_path + 'DENSITY', grid[0])

In [27]:
(ccdensity * grid[1]).sum()

13.999997492775165

In [28]:
Calculator.MrccCalculator.get_total_energy(calc_path)

{"filename": "/home/giorgiod/MRCC_interface/Diatomics/from_co_opt_ccsd_ccpvtz_68_to_77_A5_G5/A_2_G_0//run.log", "level": "error", "event": "Unable to read energy from log file.", "timestamp": "2019-06-14 09:32"}


0

In [29]:
def parse_energy_cc(log_file):
    """Parse the couple cluster energy from an MRCC output file"""
    with open(log_file,'r') as logf:
        while True:
            line=logf.readline()
            if "Final results:" in line:
                good_line=logf.readline()
                if "Total CCSD energy" in good_line:
                    for x in good_line.split(' '):
                        try:
                            float(x)
                            return (float(x))
                        except:
                            pass    

In [30]:
gl=Calculator.MrccCalculator.parse_energy_cc_Mrcc(calc_path+'/run.log')
print(gl)

-113.952850161528


In [31]:
from apdft.physics import Coulomb

In [32]:
def get_energy(proj_path,ao,go):
    return (parse_energy_cc("{}A_{}_G_{}/run.log".format(proj_path,ao,go)))

In [33]:
not 'Schetch_MRCC_readfiles.ipynb' in os.listdir('.')

False

In [34]:
get_energy(proj_path,2,1)

-113.952527910755

In [56]:
dX= 0.03779452267842504  # bohrs 
dXa=.02               # angstroms
dL=0.05  ## delta Lambda = 0.05

In [36]:
k= (get_energy(proj_path,0,1)+get_energy(proj_path,0,-1)-2*get_energy(proj_path,0,0))/dX**2

In [38]:
#### constants #####
angstorm=1.889726133921252
N_mass=25532.936684486547
centimeter=188972613.39212522
light_speed= 137.03599975303575
C_mass=21893.98614768941
O_mass=29165.124304476205
mu_nn=N_mass/2
mu_co=C_mass*O_mass/(C_mass+O_mass)

In [39]:
def to_cm(k,mu=mu_co):
    return (k/mu)**0.5*centimeter/light_speed/(2*np.pi)

In [40]:
to_cm(k) ### ok good job

2241.120735039024

In [41]:
def common_grid(go):
    return (Calculator.MrccCalculator.get_grid(None, None, proj_path+'A_0_G_{}/'.format(go)))

In [42]:
####### alchemy expantion ####  COMMON_GRID IS TO BE DEFINED FOR EACH GEOMETRY  
def get_ccdensity(ao,go):
    return Calculator.MrccCalculator.density_on_grid("{}A_{}_G_{}/DENSITY".format(proj_path,ao,go),common_grid(go)[0])

In [43]:
get_ccdensity(-2,-2)-get_ccdensity(2,-2)

array([2.94618559e-22, 3.79179046e-22, 5.17586133e-22, ...,
       6.34885971e-27, 3.95003423e-27, 2.96783422e-27])

In [44]:
get_energy(proj_path,0,1)

-113.162330773499

In [45]:
from apdft.physics import Coulomb

In [57]:
### very poor code 
co_minR=1.125614241653 # angstrom
atoms=np.asarray([6.,8.])
def coordinates(go): ### angstrom 
    return np.asarray([[0.,0.,0.],[ co_minR+go*dXa,0.,0.]])
target=np.asarray([7.,7.])

In [50]:
print(Coulomb.nuclei_nuclei(coordinates(0),atoms)/48)
print(Coulomb.nuclei_nuclei(coordinates(0),target)/49)

0.4701230591156049
0.4701230591156049


In [58]:
from apdft.physics import angstrom
angstrom


1.8897261254578281

In [68]:
ds = []
for site in coordinates(0):
    ds.append(np.linalg.norm((common_grid(0)[0] - site)*angstrom,axis=1))

# get target predictions

deltaV0 = np.zeros(len(common_grid(0)[1]))
for atomidx in range(len(atoms)):
    deltaV0 += (target[atomidx]-atoms[atomidx]) / ds[atomidx]
print (deltaV0)
def deltaV(go):
    ds = []
    for site in coordinates(go):
        ds.append(np.linalg.norm((common_grid(go)[0] - site)*angstrom,axis=1))
                #  grid, coordinates in angstrom * 1.88 -> bohr (au) for Hamiltonian calcs.
    deltaV = np.zeros(len(common_grid(go)[1]))
    for atomidx in range(len(atoms)):
        deltaV += (target[atomidx]-atoms[atomidx]) / ds[atomidx]
    return (deltaV)


[ 0.00297449  0.00292255  0.00283501 ... -0.00253396 -0.00259122
 -0.00262486]


$\tilde{\rho} = \rho^0 +\frac{1}{2}\rho' +\frac{1}{6}\rho " + \frac{1}{24}\rho ^{(3)}+ \frac{1}{120}\rho ^{(4)} $ 

## 5point coefficients <br>
0  :      0    :  1    :  0   :     0  <br>
1/12 :	−2/3 :	0 	 : 2/3 :	−1/12    for first deriv <br>
−1/12 :	4/3 :	−5/2 : 4/3 : −1/12  for second deriv <br>
−1/2 :	 1 	:    0 	:  −1    :  1/2   for third deriv <br>
 1  :	−4  :	6 	:  −4   : 1       for fourth deriv <br>
 
 

In [73]:
coeff0=np.asarray([0,0,1,0,0])
coeff1=np.asarray([1/12,-2/3,0,2/3,-1/12])
coeff2=np.asarray([-1/12 , 4/3 , -5/2 , 4/3 , -1/12])
coeff3=np.asarray([-1/2 , 1 , 0 , -1 , 1/2])
coeff4=np.asarray([1 ,-4 , 6 ,-4 , 1])
coeff_tilda=coeff0+1/2*coeff1+1/6*coeff2+1/24*coeff3+1/120*coeff4
print (coeff_tilda, np.sum(coeff_tilda))
coeff_taylor=coeff0+coeff1+1/2*coeff2+1/26*coeff3+1/24*coeff4
print (coeff_taylor, np.sum(coeff_taylor))

[ 0.01527778 -0.10277778  0.63333333  0.48055556 -0.02638889] 1.0000000000000002
[ 0.06410256 -0.12820513  0.          1.12820513 -0.06410256] 0.9999999999999998


In [77]:
def rho_tilda(go):
    rho_tilda=np.zeros(len(common_grid(go)[1]))
    for x in range(5):
        rho_tilda+=get_ccdensity(x-2,go)*coeff_tilda[x]
    return rho_tilda

In [78]:
print (rho_tilda(1))

[2.45577275e-20 2.42795474e-20 2.38214458e-20 ... 3.45432936e-26
 2.26405399e-26 1.75975215e-26]
